![bayt logo](https://images.g2crowd.com/uploads/product/image/social_landscape/social_landscape_b9320393d0092c1c2f9f5e1d78197b14/bayt-com.png)

# Bayt website data scrapping
**Note: All the links were collected on 20-12-2022**

In this notebook we will scrape all the jobs data in bayt website

In [ ]:
# import all the needed libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import threading
import requests
import random
import csv

### All my Global Variables

In [ ]:
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko', 
    'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0', 
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0', 
    'Mozilla/5.0 (Linux; Android 8.0.0; Pixel 2 XL Build/OPD1.170816.004) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Mobile Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Mobile/15E148 Safari/604.1', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.2 Safari/601.7.7'
]
fieldnames= ['job_title', 'job_type', 'job_categories', 'job_description', 'job_location', 'company_name', 'company_location', 'company_field', 'company_size', 'experience_needed', 'career_level', 'education_level', 'gender', 'salary', 'skills_and_tools', 'link']
errors_urls = [] # to collect all the urls that will fail
companies = {} # add company data so if I collect the company data before I don't make another request to get the company info

In [ ]:
# read my links csv file
df = pd.read_csv('/content/bayt_job_links_20-12-2022_1.csv', encoding='utf-16')

### Functions and methods for getting and scrapping the data

In [ ]:
# collect the company data by passing the company profile page on bayt
def get_company_data(link):
    # if I have collected the company data before get it and return it
    if companies.get(link):
        return companies.get(link)[0], companies.get(link)[1]

    # request to get the company data
    headers = {
        'User-Agent': random.choice(USER_AGENTS),
        "Accept-Encoding": "*",
        "Connection": "keep-alive"
    }
    response = requests.get(link, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')

    # start collecting the data
    company_size = None
    company_location = None
    company_block = soup.select_one('div.row.h-align-right.t-center-m')
    if company_block:
        for index, detail in enumerate(company_block.find_all('p', class_='d')):
            if index == 0:
            company_location = detail.text.strip()
            elif index == 1:
                company_size = detail.text.strip()
        companies[link] = [company_size, company_location]

    return company_size, company_location

In [ ]:
# get all the data I want from the job details page
def get_data(title, link, soup):

    # check if job still active and if not return False
    if soup.find(True, {'id': 'results_inner'}):
        return False

    
    # Get the main page blocks
    description = None
    skills = None
    job_details = None
    preferred_candidate = None
    main_blocks = soup.select('div.card-content.is-spaced')
    for block in main_blocks:
        if 'Description' in block.find('h2').text:
            description = block
        elif 'Skills' in block.find('h2').text:
            skills = block
        elif 'Details' in block.find('h2').text:
            job_details = block
        elif 'Candidate' in block.find('h2').text:
            preferred_candidate = block

            
    # Get Job Description
    job_description = None
    if description:
        job_description = description.get_text(separator=' ')


    # Get Job Skills
    skills_and_tools = None
    if skills:
        skills_and_tools = skills.get_text(separator=';')


    # Get Job Details (job_locarion - campany_field - job_categories - job_types - salary)
    job_location = None
    company_field = None
    job_categories = None
    job_types = None
    salary = None
    if job_details:
    details = job_details.find('dl').find_all('div')
    for detail in details:
        try:
            if 'Job Location' in detail.dt.text:
                job_location = detail.dd.text.strip()
            elif 'Company Industry' in detail.dt.text:
                company_field = detail.dd.text.strip()
            elif 'Job Role' in detail.dt.text:
                job_categories = detail.dd.text.strip()
            elif 'Employment Type' in detail.dt.text:
                job_types = detail.dd.text.strip()
            elif 'Salary' in detail.dt.text:
                salary = detail.dd.text.strip()
        except Exception:
            pass


    # Get Preferred Candidate (career_level - experience_needed - gender - education_level)
    career_level = None
    experience_needed = None
    gender = 'both'
    education_level = None
    if preferred_candidate:
        details = preferred_candidate.find('dl').find_all('div')
        for detail in details:
            if 'Career Level' in detail.dt.text:
                career_level = detail.dd.text.strip()
            elif 'Experience' in detail.dt.text:
                experience_needed = detail.dd.text.strip()
            elif 'Gender' in detail.dt.text:
                gender = detail.dd.text.strip()
            elif 'Degree' in detail.dt.text:
                education_level = detail.dd.text.strip()


    # Get Company Name
    company_name = soup.find('h1', {'id': 'job_title'}).parent.select_one('li').text.strip()


    # Get company location and company size 
    company_location = None
    company_size = None
    if company_name != 'Confidential Company':
        company_link = f"https://www.bayt.com{soup.find('h1', {'id': 'job_title'}).parent.find('a', class_='is-black').get('href')}"
        company_size, company_location = get_company_data(company_link)


    data = {
        'job_title': title,
        'job_type': job_types,
        'job_categories': job_categories,
        'job_description': job_description,
        'job_location': job_location,
        'company_name': company_name,
        'company_location': company_location,
        'company_field': company_field,
        'company_size': company_size,
        'experience_needed': experience_needed,
        'career_level': career_level,
        'education_level': education_level,
        'gender': gender,
        'salary': salary,
        'skills_and_tools': skills_and_tools,
        'link': link
    }
    return data

### My Main function

In [ ]:
# start collecting the data
def start(mini_df):
    # create a new csv file to append my data to
    csv_file = open(f'bayt_job_data_20-12-2022.csv', 'a+', encoding="utf-16")
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter ='~')
    writer.writeheader()
    
    # both the next variables used for preview puropose only to know the number of skipped urls and the progress of the loop
    loop_index = 1
    skipped_urls = 0
    
    # start looping throught all my links and get data
    for index, row in mini_df.iterrows():
        try:
            print(f'Dataframe Index: {index} - Loop Index {loop_index}')
            headers = {
                'User-Agent': random.choice(USER_AGENTS),
                "Accept-Encoding": "*",
                "Connection": "keep-alive"
            }
            response = requests.get(row['link'].strip(), headers=headers)

            soup =  BeautifulSoup(response.content, 'lxml')
            
            # collect my data using get_data function
            page_data = get_data(title=row['title'], link=row['link'].strip(), soup=soup)

            if page_data: # if page still exists and it returned a data add it to the csv file
                writer.writerow(page_data)
            loop_index += 1
        except Exception as e:
            skipped_urls += 1
            print(f'Skip Number {skipped_urls} - Dataframe Index: {index} - Loop Index: {loop_index} - URL: {row["link"]} \n {e}')
            errors_urls.append({
                'link': row['link'],
                'title': row['title']
            })

### Start my script

In [ ]:
start(df)